In [1]:
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
import pickle
from datasets import Dataset
from transformers import default_data_collator
from tqdm import tqdm

In [2]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-370m-hf")
model = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-370m-hf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def load_data_from_pickle(filepath='dataset/addition_dataset_text.pkl'):
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    return data['texts'],data['A'],data['Q']

def custom_data_collator(features):
    batch = default_data_collator(features)
    input_ids = [f['input_ids'] for f in features]
    input_ids = tokenizer(input_ids, padding=True, return_tensors="pt")
    batch['input_ids'] = input_ids['input_ids']
    batch['labels'] =input_ids['input_ids'] # labels same as input_ids for LM
    #batch['labels'][:,:20] = -100
    return batch

def prepare_dataset(texts,A,Q):
    train_size = int(0.8 * len(texts))
    train_set = Dataset.from_dict({
        'input_ids': texts[:train_size],
        'A': A[:train_size],
        'Q': Q[:train_size]
    })
    test_set = Dataset.from_dict({
        'input_ids': texts[train_size:],
        'A': A[train_size:],
        'Q': Q[train_size:]
    })
    return train_set, test_set
texts,A,Q = load_data_from_pickle()
train_dataset, test_dataset = prepare_dataset(texts,A,Q)


In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=40,
    per_device_train_batch_size=64,
    logging_dir='./logs',
    logging_steps=100,
    learning_rate=2e-3
)
lora_config =  LoraConfig(
        r=8,
        target_modules=["x_proj", "embeddings", "in_proj", "out_proj"],
        task_type="CAUSAL_LM",
        bias="none"
)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    train_dataset=train_dataset,
    dataset_text_field="input_ids",
    data_collator= custom_data_collator,
    max_seq_length = 128
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
trainer.train()

Step,Training Loss
100,3.267600
200,3.055500
300,3.041800
400,3.026800


KeyboardInterrupt: 

In [8]:
import xml.etree.ElementTree as ET
import re
def prsanswer(text):
        # Using regex to find the content inside the <ans> tags
    match = re.search(r'<ans>\s*(.*?)\s*</ans>', text)
    if match:
        return match.group(1).strip()

    return None  # In case there is no match

In [9]:
def calculate_accuracy(dataset, model):
    #print(decode(dataset[0]['input_ids'][:boundary]))
    corr = 0
    batch_size = 64
    for i in tqdm(range(0, len(dataset))):
        #print(i)
        input_ids = [dataset[i]['Q']+' <ans>']
        input_ids = tokenizer(input_ids, return_tensors="pt")
        predictions = model.generate(input_ids['input_ids'].to('cuda:0'), max_length=128)
        output = tokenizer.decode(predictions[0].tolist(), skip_special_tokens=True)
        output = prsanswer(output)
        corr += (output == dataset[i]['A'])
        #print(output,dataset[i]['A'],output == dataset[i]['A'])
    print(corr/len(dataset))

In [10]:
calculate_accuracy(test_dataset, model)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:16<00:00,  6.31it/s]

0.4605


In [11]:
for i in range(6,21):
    texts, A, Q= load_data_from_pickle(f'dataset/addition_dataset_text{i}.pkl')
    train_dataset, test_dataset = prepare_dataset(texts, A, Q)
    print(i, calculate_accuracy(test_dataset,model))
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:29<00:00,  6.08it/s]


0.001
6 None


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:41<00:00,  5.86it/s]


0.0
7 None


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:56<00:00,  5.61it/s]


...

0.0
18 None


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [08:18<00:00,  4.01it/s]


0.0
19 None


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [08:14<00:00,  4.05it/s]

0.0
20 None
